# Using the Indices data-set

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/Indices.ipynb)

In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")

indices<-Indices()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### fetch the last 10 day India VIX levels

In [2]:
startDate <- Sys.Date() - 10
indices$IndiaVixTimeSeries() %>%
  filter(TIME_STAMP >= startDate) %>%
  arrange(TIME_STAMP) %>%
  print(n=Inf)

# Source:     lazy query [?? x 5]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Ordered by: TIME_STAMP
  TIME_STAMP  HIGH   LOW  OPEN CLOSE
  <date>     <dbl> <dbl> <dbl> <dbl>
1 2019-08-19  17.0  14.7  16.6  16.7
2 2019-08-20  17.1  15.1  16.7  16.6
3 2019-08-21  17.3  14    16.6  17.0
4 2019-08-22  18.1  14.5  17.0  17.9
5 2019-08-23  18.3  13.7  17.9  17.3
6 2019-08-26  19.1  13.7  17.3  16.7


### fetch the latest rates across tenors

In [3]:
maxDt <- (indices$IndiaGsecTimeSeries() %>%
    summarize(MAX_TS = max(TIME_STAMP)) %>%
    collect())$MAX_TS[1]

indices$IndiaGsecTimeSeries() %>%
    filter(TIME_STAMP == maxDt) %>%
    arrange(NAME) %>%
    print()

Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:     lazy query [?? x 7]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Ordered by: NAME
  NAME  TIME_STAMP   TRI   PRI COUPON   YTM DURATION
  <chr> <date>     <dbl> <dbl>  <dbl> <dbl>    <dbl>
1 0_5   2019-08-26 2965.  842.   7.89  6.07     2.85
2 10_15 2019-08-26 2957.  908.   7.82  6.84     8.02
3 15_20 2019-08-26 2956.  899.   7.69  6.91     9.50
4 20_30 2019-08-26 2973.  894.   8.07  6.97    11.6 
5 5_10  2019-08-26 2974.  866.   7.81  6.61     5.66


### get all total-return NSE indices for the latest time_stamp

In [4]:
maxDtNse <- (indices$NseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtNse && NAME %like% '% TR%') %>%
     select(NAME) %>%
     print(n = 10)

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Groups:   NAME
   NAME                                          
   <chr>                                         
 1 NIFTY 100 TR                                  
 2 NIFTY 200 TR                                  
 3 NIFTY 50 TR                                   
 4 NIFTY 500 TR                                  
 5 NIFTY ADITYA BIRLA GROUP TR                   
 6 NIFTY ALPHA 50 TR                             
 7 NIFTY ALPHA LOW-VOLATILITY 30 TR              
 8 NIFTY ALPHA QUALITY LOW-VOLATILITY 30 TR      
 9 NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30 TR
10 NIFTY AUTO TR                                 
# … with more rows


## get start and end dates for all total-return NSE indices

In [5]:
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(NAME %like% '% TR%') %>%
     summarize(ST=min(TIME_STAMP), ET=max(TIME_STAMP)) %>%
     print(n = 10)

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:   lazy query [?? x 3]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
   NAME                                           ST         ET        
   <chr>                                          <date>     <date>    
 1 NIFTY 100 TR                                   2003-01-01 2019-08-26
 2 NIFTY 200 TR                                   2004-01-01 2019-08-26
 3 NIFTY 50 FUTURES TR INDEX                      2005-04-01 2019-08-23
 4 NIFTY 50 TR                                    1999-06-30 2019-08-26
 5 NIFTY 500 TR                                   1998-11-26 2019-08-26
 6 NIFTY ADITYA BIRLA GROUP TR                    2005-04-01 2019-08-26
 7 NIFTY ALPHA 50 TR                              2003-12-31 2019-08-26
 8 NIFTY ALPHA LOW-VOLATILITY 30 TR               2005-04-01 2019-08-26
 9 NIFTY ALPHA QUALITY LOW-VOLATILITY 30 TR       2005-04-01 2019-08-26
10 NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30 TR 2005-04-01 2019-08-26
# … with more rows


### get all BSE indices for the latest time_stamp

In [6]:
maxDtBse <- (indices$BseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$BseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtBse) %>%
     select(NAME) %>%
     print(n = 10)

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Groups:   NAME
   NAME  
   <chr> 
 1 BSE100
 2 BSE200
 3 BSE500
 4 BSEMID
 5 BSESML
 6 CARBON
 7 CPSE  
 8 DOL100
 9 DOLL30
10 DOLLEX
# … with more rows


### fetch the latest NSE NIFTY 50 constituents 

In [7]:
maxDt <- (indices$NseConstituents() %>%
          filter(NAME == "NIFTY 50") %>%
          summarize(MAX_TS = max(TIME_STAMP)) %>% 
          collect())$MAX_TS[1]

indices$NseConstituents() %>%
     filter(TIME_STAMP == maxDt & NAME == "NIFTY 50") %>%
     print(n = 10)

# Source:   lazy query [?? x 5]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
   NAME     TIME_STAMP SYMBOL     INDUSTRY           CAP_WEIGHT
   <chr>    <date>     <chr>      <chr>                   <dbl>
 1 NIFTY 50 2019-08-26 ADANIPORTS SERVICES                0.643
 2 NIFTY 50 2019-08-26 ASIANPAINT CONSUMER GOODS          1.56 
 3 NIFTY 50 2019-08-26 AXISBANK   FINANCIAL SERVICES      3.09 
 4 NIFTY 50 2019-08-26 BAJAJ-AUTO AUTOMOBILE              0.768
 5 NIFTY 50 2019-08-26 BAJAJFINSV FINANCIAL SERVICES      0.966
 6 NIFTY 50 2019-08-26 BAJFINANCE FINANCIAL SERVICES      1.73 
 7 NIFTY 50 2019-08-26 BHARTIARTL TELECOM                 1.49 
 8 NIFTY 50 2019-08-26 BPCL       ENERGY                  0.609
 9 NIFTY 50 2019-08-26 BRITANNIA  CONSUMER GOODS          0.668
10 NIFTY 50 2019-08-26 CIPLA      PHARMA                  0.563
# … with more rows


### fetch the latest BSE SENSEX constituents    

In [8]:
maxDt <- (indices$BseConstituents() %>%
          filter(NAME == "sp bse sensex") %>%
          summarize(MAX_TS = max(TIME_STAMP)) %>% 
          collect())$MAX_TS[1]

indices$BseConstituents() %>%
     filter(TIME_STAMP == maxDt & NAME == "sp bse sensex") %>%
     print(n = 10)

# Source:   lazy query [?? x 5]
# Database: Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
   NAME          TIME_STAMP   CODE SYMBOL     SECURITY_NAME                   
   <chr>         <date>      <int> <chr>      <chr>                           
 1 sp bse sensex 2019-07-01 500010 HDFC       Housing Development Finance Corp
 2 sp bse sensex 2019-07-01 500034 BAJFINANCE Bajaj Finance Ltd               
 3 sp bse sensex 2019-07-01 500112 SBIN       State Bank of India             
 4 sp bse sensex 2019-07-01 500180 HDFCBANK   HDFC Bank Ltd                   
 5 sp bse sensex 2019-07-01 500182 HEROMOTOCO Hero MotoCorp Ltd               
 6 sp bse sensex 2019-07-01 500209 INFY       Infosys Ltd                     
 7 sp bse sensex 2019-07-01 500247 KOTAKBANK  Kotak Mahindra Bank Ltd         
 8 sp bse sensex 2019-07-01 500295 VEDL       Vedanta Ltd                     
 9 sp bse sensex 2019-07-01 500312 ONGC       Oil & Natural Gas Corp Ltd      
10 sp bse sensex 2019-07-01 500325

### fetch index date ranges published on yahoo finance

In [9]:
indices$YahooFinanceTimeSeries() %>%
    group_by(NAME) %>%
    summarize(ST = min(TIME_STAMP), ET = max(TIME_STAMP)) %>%
    arrange(NAME) %>%
    print(n=Inf)

# Source:     lazy query [?? x 3]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockVizUs2]
# Ordered by: NAME
  NAME  ST         ET        
  <chr> <date>     <date>    
1 ^GSPC 1950-01-03 2019-08-09
2 ^MID  1981-01-02 2019-08-09
3 ^N225 1965-01-05 2019-08-09
4 ^NDX  1985-10-01 2019-08-09
5 ^RAG  2002-09-30 2019-08-09
6 ^RAV  2002-09-30 2019-08-09
7 ^RUA  1987-09-10 2019-08-09
8 ^RUT  1987-09-10 2019-08-09
9 ^VIX  1990-01-02 2019-08-09


### fetch index date ranges published by Barclays

In [10]:
indices$BarclaysMeta() %>%
  inner_join(indices$BarclaysTimeSeries()) %>%
  group_by(FAMILY, NAME) %>%
  summarize(ST = min(TIME_STAMP), ET = max(TIME_STAMP)) %>%
  select(FAMILY, NAME, ST, ET) %>%
  arrange(FAMILY) %>%
  print(n = Inf)

Joining, by = "TICKER"


# Source:     lazy query [?? x 4]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockVizUs2]
# Groups:     FAMILY
# Ordered by: FAMILY
    FAMILY              NAME                               ST         ET        
    <chr>               <chr>                              <date>     <date>    
  1 ARMOUR              Barclays ARMOUR CHF 10% Index      2012-11-30 2019-08-23
  2 ARMOUR              Barclays ARMOUR CHF 7% Index       1998-12-30 2019-08-23
  3 ARMOUR              Barclays ARMOUR CHF Index          2012-07-31 2019-08-23
  4 ARMOUR              Barclays ARMOUR EUR 10% Index      2012-11-30 2019-08-23
  5 ARMOUR              Barclays ARMOUR EUR 7% Index       2012-11-30 2019-08-23
  6 ARMOUR              Barclays ARMOUR EUR Index          2012-07-31 2019-08-23
  7 ARMOUR              Barclays ARMOUR II Gross USD 5% E… 2001-12-31 2019-08-23
  8 ARMOUR              Barclays ARMOUR II Gross USD 5% T… 2001-12-31 2019-08-23
  9 ARMOUR              Barclays ARMOUR II G

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)